### 추천서비스 로직 만들기

(임시)


일단 데이터 프레임으로 존재한다면.  데이터가 완성되기 전까지 

주식의 종가 = 암호화폐 종가

주식의 거례량 = SNS 감정지수

(각코인별)



날짜 - 시세, 감정지수()

### 구상
- 필요 정보
날짜별 - 시세, 감정지수()

- 추천기준이되는 정보

개인의 투자 기간 n = 예)1주/1달/3달/6달  <--- 주식과 달리 7일,   n= 7,30,90,180

주기는 2*n 로 평균이동선 측정. 왜냐면 저점에서 고점에 판다라고 생각(반주기)  그누구도 고점에사서 고점에 팔지 않기 때문

함수가 필요한 평균이동선 = 2*n

**감정지수 마찬가지 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls -l '/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/C.csv'

-rw------- 1 root root 165058 Nov  8 06:32 '/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/C.csv'


In [5]:
AAPL = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/AAPL.csv')
BABA = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/BABA.csv',encoding='cp949')
C = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/C.csv', encoding='cp949')
INTC = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/INTC.csv', encoding='cp949')
MSFT = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/MSFT.csv')
TSLA = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/TSLA.csv')
UGL = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/UGL.csv', encoding='cp949')
WFC = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/stock_market/WFC.csv', encoding='cp949')

In [6]:
AAPL.head()

,일자,시가,고가,저가,종가,BB_상한선,거래량
0,20211105,151.890,152.20,150.06,151.28,155.3499,"65,463,883"
1,20211104,151.580,152.43,150.64,150.96,155.2323,"60,394,616"
2,20211103,150.390,151.97,149.82,151.49,155.1252,"54,511,534"
3,20211102,148.660,151.57,148.65,150.02,154.9967,"69,121,987"
4,20211101,148.985,149.70,147.80,148.96,154.9146,"74,588,258"


In [7]:
BABA.head()

,일자,시가,고가,저가,종가,BB_상한선,거래량
0,20211105,164.795,164.795,158.30,158.73,192.0304,"22,098,218"
1,20211104,169.280,169.940,164.50,164.79,193.8534,"16,669,249"
2,20211103,165.360,166.650,163.11,166.24,195.6702,"13,569,682"
3,20211102,167.100,167.100,162.76,162.90,197.1079,"19,072,886"
4,20211101,165.840,171.895,165.80,170.17,198.7231,"17,609,488"
